## TODO

* setup proper initalisation of model weights
* use same drop out they used in paper
* weight the underrepresented class
* k-fold validation

## things to try

* data augmentation
 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [3]:
%matplotlib inline

In [4]:
! ls ~/data/kaggle/dont-overfit

sample_submission.csv  test.csv  test.csv.zip  train.csv


In [5]:
df_train = pd.read_csv("~/data/kaggle/dont-overfit/train.csv")

In [6]:
x = df_train.drop(columns=['target','id']).values

In [7]:
mean = np.mean(x)
std = np.std(x)

def normalize(X):
    return (X-mean)/std

In [8]:
class DofDataset(Dataset):
    def __init__(self,df, test=False):
        self.df = df
        self.test = test
            
    @property
    def tensor_labels(self):
        return torch.from_numpy(self.df.target.values).long()
    
    @property
    def tensor_data(self): #need to normalise data
        if self.test:
            return torch.from_numpy(self.df.drop(columns=['id']).values).float()
        return torch.from_numpy(self.df.drop(columns=['target', 'id']).values).float()
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        if self.test:
            return self.tensor_data[idx]
        label = self.tensor_labels[idx]
        data = self.tensor_data[idx]
        
        return label, data
        
    
        

In [9]:
# mean = np.mean(X_train)
# std = np.std(X_train)

# def standardize(x):
#     return (x-mean)/std

In [10]:
dod_ds = DofDataset(df_train)

## Metrics

In [11]:
# def accuracy(preds,labels):
#     preds_np = np.argmax(preds.detach().numpy(), axis=1)
#     return np.sum(preds_np == labels.detach().numpy())/len(preds_np)

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

## Model Defintion

In [201]:
in_size = 300
out_size = 2

model = nn.Sequential(
    nn.Linear(in_size, 600, bias=True),
    nn.SELU(),
    nn.Linear(600, 300, bias=True),
    nn.SELU(),
    nn.Linear(300, out_size, bias=True),
    nn.SELU(),
    nn.Dropout(p=0.7))
    


In [202]:
def init_normal(m):
    if type(m) == nn.Linear:
        size = m.in_features
        nn.init.normal_(m.weight,  mean=0, std=1/math.sqrt(size))

In [203]:
model.apply(init_normal)

Sequential(
  (0): Linear(in_features=300, out_features=600, bias=True)
  (1): SELU()
  (2): Linear(in_features=600, out_features=300, bias=True)
  (3): SELU()
  (4): Linear(in_features=300, out_features=2, bias=True)
  (5): SELU()
  (6): Dropout(p=0.7)
)

## Train Definition

In [210]:
lr = 1e-6
bs = 30

cross_en = nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr=lr)


In [211]:
dod_df = DataLoader(dod_ds, bs, shuffle=False)

In [212]:
def train_step(batch, train):
    labels, data = batch
    data_norm = normalize(data)
    preds = model(data_norm)
    loss = cross_en(preds, labels)
    acc = accuracy(preds,labels)
    if train:
        optim.zero_grad() # zero gradients (?? why do we do this)
        loss.backward() # calcuate gradients
        optim.step() # updated weights
    return loss.item(), acc

In [216]:
epochs = 15

In [217]:
train_loss_t = []
train_acc_t = []

In [218]:
for i in range(epochs):
    train_loss_ep = []
    train_acc_ep = []
    
    for train_batch in dod_df :
        train_loss, train_acc = train_step(train_batch, True)
        train_loss_ep.append(train_loss)
        train_acc_ep.append(train_acc)
    if i% 1 == 0:
        print(f'epoch:{i} Train Loss:  {np.mean(train_loss_ep)}, Accuracy {np.mean(train_acc_ep)}')
    train_loss_t.append(np.mean(train_loss_ep))
    train_acc_t.append(np.mean(train_acc_ep))


epoch:0 Train Loss:  0.3605426102876663, Accuracy 0.7777777910232544
epoch:1 Train Loss:  0.32222340669896865, Accuracy 0.859259307384491
epoch:2 Train Loss:  0.35040323932965595, Accuracy 0.7962962985038757
epoch:3 Train Loss:  0.3145899805757735, Accuracy 0.8370369672775269
epoch:4 Train Loss:  0.34004322025511, Accuracy 0.800000011920929
epoch:5 Train Loss:  0.3502122826046414, Accuracy 0.8148148059844971
epoch:6 Train Loss:  0.3631708323955536, Accuracy 0.7999999523162842
epoch:7 Train Loss:  0.3681585027111901, Accuracy 0.7925925850868225
epoch:8 Train Loss:  0.3528202242321438, Accuracy 0.8037037253379822
epoch:9 Train Loss:  0.34253713488578796, Accuracy 0.8259259462356567
epoch:10 Train Loss:  0.40141502353880143, Accuracy 0.7777777910232544
epoch:11 Train Loss:  0.3859183258480496, Accuracy 0.7999999523162842
epoch:12 Train Loss:  0.32698221504688263, Accuracy 0.82962965965271
epoch:13 Train Loss:  0.37334882881906295, Accuracy 0.8111111521720886
epoch:14 Train Loss:  0.380947

## Submission

In [219]:
df_test = pd.read_csv("~/data/kaggle/dont-overfit/test.csv")

In [220]:
test_ds = DofDataset(df_test, test=True)

In [221]:
test_dl = DataLoader(test_ds, batch_size=100)

In [222]:
preds = []

In [223]:
%%time
for i, batch in enumerate(test_dl):
    logits = model(batch)
    preds_np = np.argmax(logits.detach().numpy(), axis=1)
    preds.extend(preds_np)

CPU times: user 14min 10s, sys: 41.4 s, total: 14min 51s
Wall time: 14min 11s


In [224]:
df_samp = pd.read_csv("~/data/kaggle/dont-overfit/sample_submission.csv")

In [225]:
df_samp["target"] = preds

In [226]:
df_samp.to_csv("subm-8.csv", index=False)

In [227]:
! kaggle competitions submit -c dont-overfit-ii -f subm-8.csv -m "snn layer nn normalised drop 0.5, initalised with 1/2sqrt in layer, pla with train"

100%|████████████████████████████████████████| 144k/144k [00:08<00:00, 18.3kB/s]
Successfully submitted to Don't Overfit! II